# Initialize Environment

In [ ]:
import matplotlib, matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import numpy as np
from scipy.stats import norm
from scipy.io import loadmat
import glob
import pandas as pd
import datetime
import pandas_datareader.tsp as tsp
import pandas_datareader.data as pdr
from pandas_datareader.yahoo.headers import DEFAULT_HEADERS
import requests_cache
#import yfinance as yf
#yf.pdr_override() # https://pypi.org/project/yfinance/

from copy import deepcopy
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# Seed Random number generator for repeatability
np.random.seed(19690720)

plt.style.use('default')
#matplotlib.rcParams['font.family'] = 'serif'
matplotlib.rcParams.update({'font.size': 18})

# Load Data
## Data Caches

In [ ]:
session = requests_cache.CachedSession(cache_name='cache', 
                                       backend='sqlite', 
                                       expire_after=datetime.timedelta(hours=8))
session.headers = DEFAULT_HEADERS

## TSP
Load TSP data using the pandas_datareader utility. Allow for 8-hour persistent caching. 

In [ ]:
tspreader = tsp.TSPReader(start=datetime.date.today() - datetime.timedelta(days=365*25),
                          session=session)
full_tspdata = tspreader.read()

# write the TSP data, last 90 values, in reversed order
full_tspdata.iloc[:400].to_excel('tsp_shareprices.xlsx', sheet_name='TSP Data')

#print(full_tspdata)
tspdata = full_tspdata.drop(['L Income', 'L 2025', 'L 2030', 'L 2035', 'L 2040', 'L 2045', 'L 2050', 'L 2055', 'L 2060', 'L 2065'],axis=1).iloc[::-1]

## Vanguard

In [ ]:
try:
    tickers
except NameError:    
    tickers = [ 'VTSAX', 'VSGAX', 'VGSLX', 'VHYAX', 'VTMSX', 'VTCLX', 'VTMFX', 'VBTLX', 'VWAHX', 'VGSTX', 'VIHAX', 'VWIGX' ]

data = pdr.DataReader(
        # tickers list or string as well
        tickers, 'yahoo', session=session
    )
data.head()
#rev_data = data.iloc[::-1]
#rev_data.iloc[:800].to_excel('vg_shareprices.xlsx', sheet_name='ETF Data')

# Derived Data

In [ ]:
ddata = data['Close']
ddata.head()